In [1]:
import os
import numpy as np # import numerical python
import pandas as pd # import pandas
import scipy as sp
import xarray as xr # import xarray
import struct # import for importing binary
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.special import erf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.font_manager
from matplotlib import cm
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import sys
sys.path.append("/home/jbeckwith/Documents/Dropbox (Cambridge University)/Chemistry/Lee/Code/Python/CommonFunctions") 
import GaussExp_Distribution
import MVChangePoint
import DHOF_Distribution
import DHOA_Distribution
from PlottingFunctions import Plotter
plots = Plotter(poster=True)
GED = GaussExp_Distribution.gaussexp_dist
MVCP = MVChangePoint.MVCP()

In [ ]:
data = pd.read_csv('Testing_Gradient/20231204_GradTesting_A=500_B=1_Tau1_1ns_Tau2_2ns.csv', index_col=0)

In [ ]:
data_2 = pd.read_csv('Testing_Gradient/20231204_GradTesting_A=500_B=1_Tau1_10ns_Tau2_20ns.csv', index_col=0)

In [ ]:
plt.hist(data_2.Q_k.values/np.nanmax(data_2.Q_k.values), 100);
plt.hist(data.Q_k.values/np.nanmax(data.Q_k.values), 100);

#plt.hist(data.GradientAfter.values, 100);
plt.show()

In [ ]:
plt.scatter(data.CP_location.values, data.IntGradientBefore.values*data.Q_k.values*np.abs(data.IntGradientAfter.values))

In [ ]:
#plt.scatter(data.CP_location.values, data.IntGradientAfter.values+data.IntGradientBefore.values)
#plt.scatter(data.CP_location.values, np.abs(data.IntGradientAfter.values)+data.IntGradientBefore.values)

plt.scatter(data.CP_location.values, data.Q_k.values*data.IntGradientBefore.values)
#plt.scatter(data.CP_location.values, data.IntGradientAfter.values/data.IntGradientBefore.values)

#plt.scatter(data.CP_location.values, data.SumGradientBefore.values)

In [ ]:
plt.scatter(data.CP_location.values, data.IntGradientAfter.values)#plt.scatter(data.CP_location.values, )

In [ ]:
value = 100
plt.hist(data[data.IntGradientBefore > value].CP_location.values, 100);
#plt.hist(data[data.GradientBefore+np.abs(data.GradientAfter) < value].CP_location.values, 100);
plt.show()

In [ ]:
data[data.IntGradientBefore > value].CP_location.values

In [ ]:
tau1 = 1
A = 500
B = 1
amps = 1
sigma = 0.1

tau2list = np.logspace(np.log10(tau1/10.), np.log10(tau1*10), 100)
ntests = 100
tau2 = 2
    
mydist_1 = GED(tau1, amps, sigma)
mydist_2 = GED(tau2, amps, sigma)
nsignal = int(A/2)
CPL = A/2
data = np.zeros([A, B, ntests])
for i in np.arange(B):
    signal_1 = (mydist_1.rvs(size=(nsignal, ntests))) # get signal
    signal_2 = (mydist_2.rvs(size=(nsignal, ntests))) # get signal
    data[:, i, :] = np.vstack([signal_1, signal_2])
    
for test in np.arange(ntests):
    test_data = data[:, :, test]
    CP_list = MVCP.NPCP_EDivisive(test_data)
    CP_list = np.hstack([np.full([CP_list.shape[0], 1], test+1), CP_list])
    if test == 0:
        overallCPlist = CP_list
    else:
        overallCPlist = np.vstack([overallCPlist, CP_list])
    if test % 1 == 0:
        print(str(test)+' of '+str(ntests)+', from tau '+str(np.around(tau2, 2))+' done.')
columns = ['test', 'CP_location', 'Significance', 'Q_k', 'SumGradientBefore', 'SumGradientAfter', 'IntGradientBefore', 'IntGradientAfter']
oCP_tosave = pd.DataFrame(data=overallCPlist, columns=columns)
tautosave = str(np.around(tau2, 2)).replace('.', 'p')
oCP_tosave.to_csv('Testing_Gradient/20231204_GradTesting_A='+str(A)+'_B='+str(B)+'_Tau1_1ns_Tau2_'+tautosave+'ns.csv')
pd.DataFrame(data=np.full(ntests, CPL), columns=['TPs']).to_csv('Testing_Gradient/20231204_TruePositives_A='+str(A)+'_B='+str(B)+'Tau1_1ns_Tau2_'+tautosave+'ns.csv')

In [ ]:
# do MV gaussian tests

loc1 = 1
A = 500
B = 20
amps = 1
ntests = 100
loc2 = 2
    
nsignal = int(A/2)
CPL = A/2
data = np.zeros([A, B, ntests])
for t in np.arange(ntests):
    signal_1 = np.sort(np.random.normal(size=(nsignal, B), loc=loc1, scale=1), axis=1) # get signal
    signal_2 = np.sort(np.random.normal(size=(nsignal, B), loc=loc2, scale=1), axis=1) # get signal
    data[:, :, t] = np.vstack([signal_1, signal_2])
    
for test in np.arange(ntests):
    test_data = data[:, :, test]
    CP_list = MVCP.NPCP_EDivisive(test_data)
    CP_list = np.hstack([np.full([CP_list.shape[0], 1], test+1), CP_list])
    if test == 0:
        overallCPlist = CP_list
    else:
        overallCPlist = np.vstack([overallCPlist, CP_list])
    if test % 1 == 0:
        print(str(test)+' of '+str(ntests)+', from loc '+str(np.around(loc2, 2))+' done.')
columns = ['test', 'CP_location', 'Significance', 'Q_k', 'SumGradientBefore', 'SumGradientAfter', 'IntGradientBefore', 'IntGradientAfter']
oCP_tosave = pd.DataFrame(data=overallCPlist, columns=columns)
oCP_tosave.to_csv('Testing_Gradient/20231204_NEWSIGTEST_GradTesting_A='+str(A)+'_B='+str(B)+'_MVGaussian_loc1_1_loc2_2.csv')
pd.DataFrame(data=np.full(ntests, CPL), columns=['TPs']).to_csv('Testing_Gradient/20231204_TruePositives_A='+str(A)+'_B='+str(B)+'_MVGaussian_loc1_1_loc2_2.csv')

In [14]:
tau2list = np.geomspace((tau1/9.), (tau1*9), 50)
tau2list

array([ 1.11111111,  1.21536343,  1.32939744,  1.45413093,  1.5905678 ,
        1.73980614,  1.90304708,  2.08160445,  2.27691533,  2.49055166,
        2.72423287,  2.97983971,  3.25942939,  3.56525215,  3.89976938,
        4.26567338,  4.66590908,  5.10369774,  5.58256283,  6.10635844,
        6.67930027,  7.30599956,  7.99150021,  8.7413194 ,  9.56149194,
       10.45861887, 11.43992062, 12.51329504, 13.68738107, 14.97162816,
       16.37637242, 17.91291974, 19.59363682, 21.43205069, 23.44295758,
       25.642542  , 28.04850702, 30.68021673, 33.55885209, 36.70758143,
       40.15174687, 43.91906832, 48.03986657, 52.54730731, 57.47766809,
       62.87063026, 68.76959835, 75.22204945, 82.27991523, 90.        ])

In [17]:
tau1 = 10
A = 500
B = 1
amps = 1
sigma = 0.1

tau2list = np.geomspace((tau1/12.), (tau1*12), 100)
ntests = 100

for tau2 in tau2list:
    
    mydist_1 = GED(tau1, amps, sigma)
    mydist_2 = GED(tau2, amps, sigma)
    nsignal = int(A/2)
    CPL = A/2
    data = np.zeros([A, B, ntests])
    for i in np.arange(B):
        signal_1 = (mydist_1.rvs(size=(nsignal, ntests))) # get signal
        signal_2 = (mydist_2.rvs(size=(nsignal, ntests))) # get signal
        data[:, i, :] = np.vstack([signal_1, signal_2])

    print('data generated')
    coords = [np.arange(0, A), np.arange(0, B), np.arange(0, ntests)]
    dims = ['t', 'B', 'test']
    data_tosave = xr.DataArray(data=data, coords=coords, dims=dims)
    tautosave = str(np.around(tau2, 3)).replace('.', 'p')
    data_tosave.to_netcdf('Lifetime_forPlot/TauTesting_tau1_10ns_tau2_'+tautosave+'ns_Data.nc')
    for test in np.arange(ntests):
        test_data = data[:, :, test]
        CP_list = MVCP.NPCP_EDivisive(test_data)
        CP_list = np.hstack([np.full([CP_list.shape[0], 1], test+1), CP_list])
        if test == 0:
            overallCPlist = CP_list
        else:
            overallCPlist = np.vstack([overallCPlist, CP_list])
        if test % 5 == 0:
            print(str(test)+' of '+str(ntests)+', from tau '+str(np.around(tau2, 2))+' done.')
    oCP_tosave = pd.DataFrame(data=overallCPlist, columns=['test', 'CP_location', 'Significance'])
    oCP_tosave.to_csv('Lifetime_forPlot/20231204_TauTesting_Tau1_10ns_Tau2_'+tautosave+'ns.csv')
    #pd.DataFrame(data=np.full(ntests, CPL), columns=['TPs']).to_csv('Lifetime_forPlot/20231204_TruePositives_Tau1_10ns_Tau2_'+tautosave+'ns.csv')

data generated
0 of 100, from tau 0.83 done.
5 of 100, from tau 0.83 done.
10 of 100, from tau 0.83 done.
15 of 100, from tau 0.83 done.
20 of 100, from tau 0.83 done.
25 of 100, from tau 0.83 done.
30 of 100, from tau 0.83 done.
35 of 100, from tau 0.83 done.
40 of 100, from tau 0.83 done.
45 of 100, from tau 0.83 done.
50 of 100, from tau 0.83 done.
55 of 100, from tau 0.83 done.
60 of 100, from tau 0.83 done.
65 of 100, from tau 0.83 done.
70 of 100, from tau 0.83 done.
75 of 100, from tau 0.83 done.
80 of 100, from tau 0.83 done.
85 of 100, from tau 0.83 done.
90 of 100, from tau 0.83 done.
95 of 100, from tau 0.83 done.
data generated
0 of 100, from tau 0.88 done.
5 of 100, from tau 0.88 done.
10 of 100, from tau 0.88 done.
15 of 100, from tau 0.88 done.
20 of 100, from tau 0.88 done.
25 of 100, from tau 0.88 done.
30 of 100, from tau 0.88 done.
35 of 100, from tau 0.88 done.
40 of 100, from tau 0.88 done.
45 of 100, from tau 0.88 done.
50 of 100, from tau 0.88 done.
55 of 100, fr

KeyboardInterrupt: 

In [2]:
S = 1.25
nu0 = 20
sigma1 = 0.2
omega = 0.4
sigma2list = np.geomspace((sigma1/5.), (sigma1*5), 50)
ntests = 100
A = 500
B = 100
nsignal = int(A/2)
CPL = A/2

for sigma2 in sigma2list:
    DHO_1 = DHOF_Distribution.DHOF_dist(S, nu0, sigma1, omega)
    DHO_2 = DHOF_Distribution.DHOF_dist(S, nu0, sigma2, omega)

    data_dho = np.zeros([A, B, ntests])
    for i in np.arange(B):
        signal_1 = (DHO_1.rvs(size=(nsignal, ntests))) # get signal
        signal_2 = (DHO_2.rvs(size=(nsignal, ntests))) # get signal
        data_dho[:, i, :] = np.vstack([signal_1, signal_2])
 
    data_wavelength = 1e4/data_dho
    bins = np.linspace(480, 600, 51)
    data_wavelength_spec = np.zeros([A, len(bins)-1, ntests])
    for test in np.arange(ntests):
        for i in np.arange(A):
            data_wavelength_spec[i, :, test] = np.histogram(data_wavelength[i, :, test], bins=bins)[0]    

    print('spectra simulated')
    coords = [np.arange(0, A), np.arange(0, 50), np.arange(0, ntests)]
    dims = ['t', 'wavelength', 'test']
    sigmatosave = str(np.around(sigma2, 3)).replace('.', 'p')
    data_tosave = xr.DataArray(data=data_wavelength_spec, coords=coords, dims=dims)
    data_tosave.to_netcdf('Spectrum_forPlot/SpectralTesting_Data_Sigma1_0p2kK_Sigma2_'+sigmatosave+'kk_Data.nc')
    
    for test in np.arange(ntests):
        CP_list = MVCP.NPCP_EDivisive(data_wavelength_spec[:, :, test])
        CP_list = np.hstack([np.full([CP_list.shape[0], 1], test+1), CP_list])
        if test == 0:
            overallCPlist = CP_list
        else:
            overallCPlist = np.vstack([overallCPlist, CP_list])
        if test % 5 == 0:
            print(str(test)+' of '+str(ntests)+', from sigma '+str(np.around(sigma2, 2))+' done.')
    oCP_tosave = pd.DataFrame(data=overallCPlist, columns=['test', 'CP_location', 'Significance'])
    oCP_tosave.to_csv('Spectrum_forPlot/20231204_SpectralTesting_Sigma1_0p2kK_Sigma2_'+sigmatosave+'kK.csv')
    #pd.DataFrame(data=np.full(ntests, CPL), columns=['TPs']).to_csv('Spectrum_forPlot/20231204_TruePositives_Sigma1_0p2kK_Sigma2_'+sigmatosave+'kK.csv')

spectra simulated
0 of 100, from sigma 0.04 done.
5 of 100, from sigma 0.04 done.
10 of 100, from sigma 0.04 done.
15 of 100, from sigma 0.04 done.
20 of 100, from sigma 0.04 done.
25 of 100, from sigma 0.04 done.
30 of 100, from sigma 0.04 done.
35 of 100, from sigma 0.04 done.
40 of 100, from sigma 0.04 done.
45 of 100, from sigma 0.04 done.
50 of 100, from sigma 0.04 done.
55 of 100, from sigma 0.04 done.
60 of 100, from sigma 0.04 done.
65 of 100, from sigma 0.04 done.
70 of 100, from sigma 0.04 done.
75 of 100, from sigma 0.04 done.
80 of 100, from sigma 0.04 done.
85 of 100, from sigma 0.04 done.
90 of 100, from sigma 0.04 done.
95 of 100, from sigma 0.04 done.
spectra simulated
0 of 100, from sigma 0.04 done.
5 of 100, from sigma 0.04 done.
10 of 100, from sigma 0.04 done.
15 of 100, from sigma 0.04 done.
20 of 100, from sigma 0.04 done.
25 of 100, from sigma 0.04 done.
30 of 100, from sigma 0.04 done.
35 of 100, from sigma 0.04 done.
40 of 100, from sigma 0.04 done.
45 of 100, 

In [ ]:
fig, axs = plots.two_column_plot(ncolumns=2, widthratio=[1,1], height=3)

lwg=0.25

axs[0] = plots.line_plot(axs[0], np.arange(0, len(data)), np.mean(data, axis=1), xaxislabel='laser pulse', yaxislabel='photon arrival time')
axs[1] = plots.line_plot(axs[1], t, Q, xaxislabel='laser pulse', yaxislabel='data difference metric')

#axs[0].plot(t, Q, color='k', lw=1, label='Q metric')
axs[1].vlines(t[np.argmax(Q)], 0, np.nanmax(Q)*2, lw=0.75, color='r', label=r'argmax($\hat{\mathsf{Q}}$)')
axs[1].vlines(CPL, 0, np.nanmax(Q)*2, lw=1, ls='--', color='k', label='true CP location')
axs[1].legend(loc='upper left', frameon=False)
axs[1].set_xlim([np.nanmin(t), np.nanmax(t)])
axs[1].set_ylim([0, np.nanmax(Q)+(0.05*np.nanmax(Q))])
axs[0].grid(True,which="both",ls="--",c='gray', lw=lwg, alpha=0.25)  

#axs[1].plot(np.arange(0, len(data)), np.mean(data, axis=1), color='k', lw=1)
axs[0].vlines(CPL, 0, np.nanmax(Q)*2, lw=1, ls='--', color='k', label='true CP location')
axs[0].hlines(tau1, xmin=0, xmax=len(data)/2, color='r', lw=1.25)
axs[0].hlines(tau2, xmin=len(data)/2, xmax=len(data), color='r', lw=1.25)

axs[0].set_xlim([0, len(data)])
axs[0].set_ylim([np.min(data), np.max(np.mean(data, axis=1))*1.1])
axs[1].grid(True,which="both",ls="--",c='gray', lw=lwg, alpha=0.25)
plt.tight_layout()
plt.savefig('20231124_ChangePoint_Lifetime.svg', transparent=True, dpi=600)

In [ ]:
x = np.linspace(17, 22, 10000)
y = DHO_1.fluordispharmonic(x)

x = 1e4/x
fig, axs = plots.one_column_plot()
axs = plots.line_plot(axs, x, y, xaxislabel='wavelength/nm', yaxislabel='fluorescence', label='unstructured')


x = np.linspace(17, 22, 10000)
y = DHO_2.fluordispharmonic(x)

x = 1e4/x

axs = plots.line_plot(axs, x, y, xaxislabel='wavelength/nm', yaxislabel='fluorescence', label='structured', ls='--')
axs.set_ylim([0, 1.5])
plt.legend(loc='best')

plt.savefig('20231124_Fluo.svg', transparent=False, dpi=600)

In [ ]:
A = 500
B = 100

nsignal = int(A/2)
CPL = A/2


data_dho = np.zeros([A, B])
for i in np.arange(B):
    signal_1 = (DHO_1.rvs(size=nsignal)) # get signal
    signal_2 = (DHO_2.rvs(size=nsignal)) # get signal
    d = np.hstack([signal_1, signal_2])
    data_dho[:, i] = np.hstack([signal_1, signal_2])

In [ ]:
data_wavelength = 1e4/data_dho

In [ ]:
data_wavelength.shape

In [ ]:
bins = np.linspace(480, 582, 51)

In [ ]:
data_wavelength_spec = np.zeros([A, len(bins)-1])
for i in np.arange(A):
    data_wavelength_spec[i, :] = np.histogram(data_wavelength[i, :], bins=bins)[0]

In [ ]:
plt.plot(bins[:-1], data_wavelength_spec[300, :])

In [ ]:
alpha = 1
t = np.arange(2, len(data_wavelength_spec)-2)
Q_wavelength = MVCP.EmpiricalDivergence(data_wavelength_spec, t, alpha)

In [ ]:
fig, axs = plots.two_column_plot(ncolumns=2, widthratio=[1,1], height=3)

lwg=0.25

x = bins[:-1]
y = np.arange(0, 500)

xv, yv = np.meshgrid(y, x, indexing='ij')
z = data_wavelength_spec
axs[0].contourf(xv,yv,z)

for c in axs[0].collections:
    c.set_edgecolor("face")
axs[1] = plots.line_plot(axs[1], t, Q_wavelength, xaxislabel='time point', yaxislabel='data difference metric')

#axs[0].plot(t, Q, color='k', lw=1, label='Q metric')
axs[1].vlines(t[np.argmax(Q_wavelength)], 0, np.nanmax(Q_wavelength)*2, lw=0.75, color='r', label=r'argmax($\hat{\mathsf{Q}}$)')
axs[1].vlines(CPL, 0, np.nanmax(Q_wavelength)*2, lw=1, ls='--', color='k', label='true CP location')
axs[1].legend(loc='upper left', frameon=False)
axs[1].set_xlim([np.nanmin(t), np.nanmax(t)])
axs[1].set_ylim([0, np.nanmax(Q_wavelength)+(0.05*np.nanmax(Q_wavelength))])

#axs[1].plot(np.arange(0, len(data)), np.mean(data, axis=1), color='k', lw=1)
axs[0].vlines(CPL, 0, np.nanmax(Q_wavelength)*2, lw=1, ls='--', color='k', label='true CP location')
axs[0].hlines(np.mean(data_wavelength[:250, :]), xmin=0, xmax=len(data)/2, color='r', lw=1.25)
axs[0].hlines(np.mean(data_wavelength[250:, :]), xmin=len(data)/2, xmax=len(data), color='r', lw=1.25)

axs[0].set_xlabel('time point', fontsize=8)
axs[0].set_ylabel('wavelength', fontsize=8)
axs[0].set_xlim([0, len(data)])
axs[0].set_ylim([np.min(yv), np.max(np.mean(yv, axis=1))])
axs[1].grid(True,which="both",ls="--",c='gray', lw=lwg, alpha=0.25)
plt.tight_layout()
plt.savefig('20231124_ChangePoint_Spectrum.svg', transparent=True, dpi=600)

In [ ]:
plt.contourf(xv,yv,z)

In [ ]:
def bhattacharyya_distance(x, distribution1, distribution2):
    """ Estimate Bhattacharyya Distance (between General Distributions)
    
    Args:
        distribution1: a sample distribution 1
        distribution2: a sample distribution 2
    
    Returns:
        Bhattacharyya distance
    """
    DB = np.trapz(np.sqrt(distribution1*distribution2), x=x)
    BD = -np.log(DB)
    if BD < 0:
        BD = 0
    return BD

In [ ]:
def exppdf(x, lambda_val):
     # exp function
     # outputs normalised exp lambda*exp(-lambda*x)
     # ================INPUTS============= 
     # x ix data points to evaluate pdf
     # lambda_val is exponential rate parameter
     # ================OUTPUT============= 
     # pdf, simulated signal
    pdf = np.multiply(lambda_val, np.exp(np.multiply(-lambda_val, x))) # make exponential pdf
    return pdf # return the exponential pdf

In [ ]:
def gauss(x, sigma, mu):
     # gauss function
     # outputs normalised gaussian (1/(sqrt(2*pi)*sigma))*(exp(-0.5*((x-mu)^2 / sigma^2)))
     # ================INPUTS============= 
     # x ix data points to evaluate pdf
     # sigma is sigma of gaussian
     # mu is centre of gaussian
     # ================OUTPUT============= 
     # pdf, simulated signal
    prefactor = np.divide(1., np.multiply(sigma, np.sqrt(np.multiply(2., np.pi)))) # get normalisation prefactor
    expterm = np.exp(np.multiply(-0.5, np.divide(np.square(np.subtract(x, mu)), np.square(sigma)))) # get exponential term
    pdf = np.multiply(prefactor, expterm) # make gaussian pdf
    return pdf # return the gaussian pdf

In [ ]:
def expgauss(x, tau, sigma):
    # expgauss function
    # outputs normalised sum-of-exponentials convolved with gaussians 
    # ================INPUTS============= 
    # x ix data points to evaluate pdf
    # tau is one lifetime
    # sigma is sigma of gaussian to convolve with exponential
    # ================OUTPUT============= 
    # simulated signal
    ksm = np.tile(np.divide(1., tau), (1, 1))  # make matrix of ks
    erfterm = 1. + erf(np.divide(np.subtract(x, np.multiply(ksm.T, np.square(sigma))), np.sqrt(2)*sigma))
    gaussterm = np.exp(np.divide(np.multiply(np.square(ksm.T), np.square(sigma)), 2.))
    return np.nansum(np.multiply(np.divide(np.multiply(1., ksm), 2.).T, np.multiply(np.multiply(np.exp(np.multiply(-ksm.T, x)), gaussterm), erfterm)), axis=0)

In [ ]:
e_duration1 = np.random.exponential(scale=0.39, size=100000) # 0.39 and 0.25 are lifetime blinking time constants
e_duration2 = np.random.exponential(scale=0.25, size=100000)
plt.hist(e_duration1, 100, alpha=0.7, density=True); plt.hist(e_duration2, 100, alpha=0.7, density=True); plt.show(block=False)

In [ ]:
x = np.linspace(-2, 200, 100000);
P = expgauss(x, 39, 0.3)
Q = expgauss(x, 19, 0.3)
plt.plot(x, P)
plt.plot(x, Q)
plt.yscale('log')
plt.ylim([1e-3, 0.06])
print(bhattacharyya_distance(x, P, Q))

In [ ]:
BDists = np.zeros(100)

x = np.logspace(np.log10(1e-3), np.log10(100), 100000);
for i in enumerate(np.arange(2, 102)):
    Q = exppdf(x, 1./i[1])
    BDists[i[0]] = bhattacharyya_distance(x, P, Q)


In [ ]:
plt.plot(BDists, label='Exponential')
plt.plot(BDists_Gauss, label='Gaussian')
plt.ylabel('Bhattacharyya distance')
plt.yscale('log')
plt.legend(loc='best')
plt.show(block=False)

In [ ]:
x2 = np.linspace(-10, 20, 100000)
P2 = gauss(x2, 1, 1)
Q2 = gauss(x2, 1, 2)
plt.plot(x2, P2)
plt.plot(x2, Q2)

BD = bhattacharyya_distance(x2, P2, Q2)
print(BD)

In [ ]:
BDists_Gauss = np.zeros(100)

for i in enumerate(np.arange(1, 101)):
    BDists_Gauss[i[0]] = 0.25*(((0-i[1])**2)/(1**2 + 1**2)) + 0.5*np.log((1**2 + 1**2)/(2*1*1))


In [ ]:
wavenumbers = np.linspace(10, 30, 100000)
S = 2
nu0 = 20
sigma = 0.5
omega = 1

In [ ]:
DHDF = DHOF_Distribution.DHOF_dist(S, nu0, sigma, omega)
DHDF_2 = DHOF_Distribution.DHOF_dist(S, nu0, sigma, omega+0.5)

DHDA = DHOA_Distribution.DHOA_dist(S, nu0, sigma, omega)

In [ ]:
wavenumbers = np.linspace(10, 30, 2000)

plt.hist(DHDF.rvs(size=200), bins=wavenumbers, color='red', alpha=0.5, density=True);
plt.plot(wavenumbers, DHDF.pdf(wavenumbers), color='red');
plt.hist(DHDF_2.rvs(size=200), bins=wavenumbers, color='blue', alpha=0.5, density=True);
plt.plot(wavenumbers, DHDF_2.pdf(wavenumbers), color='blue');
plt.show(block=False)

In [ ]:
A = 200
B = 2
mydist_1 = DHDF
mydist_2 = DHDF_2
nsignal = int(A/2)
CPL = A/2


data = np.zeros([A, B])
for i in np.arange(A):
    if i < CPL:
        data[i, :] = np.sort(DHDF.rvs(size=B))
    else:
        data[i, :] = np.sort(DHDF_2.rvs(size=B))

In [ ]:
alpha = 1
t = np.arange(2, len(data)-2)
Q = MVCP.EmpiricalDivergence(data, t, alpha)

In [ ]:
plt.rcParams['figure.figsize'] = [8, 3.5]
plt.rcParams['font.size'] = 9


fig, axs= plt.subplots(1, 2)
lwg=0.25
axs[0].plot(t, Q, color='k', lw=1, label='Q metric')
axs[0].vlines(t[np.argmax(Q)], 0, np.nanmax(Q)*2, lw=0.75, color='r', label=r'argmax($\hat{\mathsf{Q}}$)')
axs[0].vlines(CPL, 0, np.nanmax(Q)*2, lw=1, ls='--', color='k', label='true CP location')
axs[0].legend(loc='upper left', frameon=False)
axs[0].set_xlabel('time step')
axs[0].set_ylabel(r'$\hat{\mathsf{Q}}$')
axs[0].set_xlim([np.nanmin(t), np.nanmax(t)])
axs[0].set_ylim([0, np.nanmax(Q)+(0.05*np.nanmax(Q))])
axs[0].grid(True,which="both",ls="--",c='gray', lw=lwg, alpha=0.25)  

axs[1].plot(np.arange(0, len(data)), np.std(data, axis=1), color='k', lw=1)
axs[1].vlines(CPL, 0, np.nanmax(Q)*2, lw=1, ls='--', color='k', label='true CP location')
axs[1].hlines(sigma, xmin=0, xmax=len(data)/2, color='r', lw=1.25)
axs[1].hlines(sigma+0.5, xmin=len(data)/2, xmax=len(data), color='r', lw=1.25)

axs[1].set_xlabel('time step')
axs[1].set_ylabel(r'$\sigma$ of '+str(A)+' by '+str(B)+' matrix')
axs[1].set_xlim([0, len(data)])
axs[1].set_ylim([np.min(np.std(data, axis=1)), np.max(np.std(data, axis=1))*1.1])
axs[1].grid(True,which="both",ls="--",c='gray', lw=lwg, alpha=0.25)
plt.tight_layout()

In [ ]:
x = np.logspace(-3, 1.5, 10000)
y = (0.39)*np.exp(-0.39*x)
plt.plot(x,y)
yh = np.random.exponential(1./0.39, 1000)
plt.hist(yh, bins=100, alpha=0.5, density=True)
plt.ylim(1e-3, 0.5)
plt.xscale('log');plt.yscale('log');
plt.grid(alpha=0.25, ls='--', lw=0.5)
plt.xlim(1e-3, np.power(10, 1.5))
plt.show()